In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
DATA_PATH=r"D:\cricket" # path where all video folder are there are there 

In [3]:
actions=os.listdir(DATA_PATH)
actions

['Cover Drive', 'Helicopter', 'Late Cut', 'Pull', 'Straight Drive', 'Sweep']

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils #Draw utilities 

In [5]:
def multiple_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #Color convertion 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [7]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        video_list=os.listdir(os.path.join(DATA_PATH, action))
        for file_name in video_list:
            path=os.path.join(DATA_PATH, action, file_name)
            cap=cv2.VideoCapture(path)
            store_path=os.path.join("D:\Cricket_images",action) # creating path to store extracted keypoints from every frame of particular video
            os.makedirs(store_path) # creating the directory
            ret, frame = cap.read()
            frame_num=0
            while ret and cap.isOpened() and frame_num<600:        

                # Make detections
                image, results = multiple_detection(frame, holistic)
                # 2. Prediction logic
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(store_path, str(frame_num))
                np.save(npy_path, keypoints)
                frame_num+=1

                # Read feed
                ret, frame = cap.read()
            
            cap.release()
            cv2.destroyAllWindows()
 
    